#### Data Wrangling Techniques Applied So Far The following Python code snippets (using the pandas library) illustrate the data preparation steps completed for RQ1 and RQ2.

In [1]:
import pandas as pd
import numpy as np

# Assuming 'pr_df' is pull_request, 'commit_df' is pr_commit_details
# and 'repo_df' is repository, etc.

# Sample data to demonstrate joins and feature creation
pr_data = {
    'id': [1, 2, 3, 4], 
    'repo_id': [101, 101, 102, 102],
    'agent': [True, False, True, False],
    'merged_at': ['2025-01-01', None, '2025-01-05', None],
    'state': ['merged', 'closed', 'merged', 'closed']
}
pr_df = pd.DataFrame(pr_data)

commit_data = {
    'pr_id': [1, 1, 2, 3, 4],
    'additions': [10, 5, 100, 20, 50],
    'deletions': [2, 1, 50, 5, 10],
    'file_path': ['src/main.py', 'src/util.py', 'docs/README.md', 'tests/test.py', 'src/func.js']
}
commit_df = pd.DataFrame(commit_data)

comments_data = {
    'pr_id': [1, 2, 3, 4],
    'total_comments': [3, 10, 1, 5]
}
comments_df = pd.DataFrame(comments_data)


## A. RQ2: Computing Patch Size and Total Comments
# 1. Compute Patch Size (total changes) per PR
patch_size_df = commit_df.groupby('pr_id').agg(
    total_additions=('additions', 'sum'),
    total_deletions=('deletions', 'sum')
).reset_index()
patch_size_df['patch_size'] = patch_size_df['total_additions'] + patch_size_df['total_deletions']

# 2. Join patch size with comment counts
rq2_df = pr_df.merge(patch_size_df[['pr_id', 'patch_size']], left_on='id', right_on='pr_id')
rq2_df = rq2_df.merge(comments_df, left_on='id', right_on='pr_id', suffixes=('_pr', '_comm'))
rq2_df = rq2_df[['id', 'agent', 'patch_size', 'total_comments']].rename(columns={'id': 'pr_id'})

# Example of RQ2 wrangling result
print("RQ2 Wrangling Snapshot:")
print(rq2_df.head())

## B. RQ1: Defining Acceptance and Categorizing File Paths
# 1. Define PR Acceptance (Target Variable)
pr_df['accepted'] = pr_df['merged_at'].notna() # merged_at != NULL -> accepted 

# 2. Extract and Categorize File Paths
def categorize_path(path):
    """Categorizes file path into high-level groups."""
    if path.startswith('src/'):
        return 'Source Code'
    elif path.startswith('docs/'):
        return 'Documentation'
    elif path.startswith('tests/'):
        return 'Test Files'
    else:
        return 'Other'

commit_df['path_category'] = commit_df['file_path'].apply(categorize_path)

# 3. Join and Aggregate for RQ1 analysis
# We aggregate the categories modified by each PR to get the main file type modified
# Note: A PR can modify multiple file types. For simplicity here, we group by PR and its categories.
rq1_data = commit_df.merge(pr_df[['id', 'agent', 'accepted']], left_on='pr_id', right_on='id')
rq1_df = rq1_data.groupby(['agent', 'path_category']).agg(
    total_prs=('id', 'nunique'),
    accepted_prs=('accepted', 'sum')
).reset_index()
rq1_df['acceptance_rate'] = rq1_df['accepted_prs'] / rq1_df['total_prs']

# Example of RQ1 wrangling result (aggregated counts)
print("\nRQ1 Wrangling Snapshot (Aggregated):")
print(rq1_df.head())

RQ2 Wrangling Snapshot:
   pr_id  agent  patch_size  total_comments
0      1   True          18               3
1      2  False         150              10
2      3   True          25               1
3      4  False          60               5

RQ1 Wrangling Snapshot (Aggregated):
   agent  path_category  total_prs  accepted_prs  acceptance_rate
0  False  Documentation          1             0              0.0
1  False    Source Code          1             0              0.0
2   True    Source Code          1             2              2.0
3   True     Test Files          1             1              1.0


#### RQ2: Relationship between Patch Size and Review Comments
##### The analysis modeled the relationship between Patch Size (total additions + deletions) and the Number of Review Comments. We used log-transformations for both variables to linearize the relationship and address heavy skewness

In [2]:
import scipy.stats as stats
import statsmodels.api as sm

# Use wrangled RQ2 data for modeling
# Apply log transformation (add 1 to handle zero values)
rq2_df['log_patch_size'] = np.log1p(rq2_df['patch_size'])
rq2_df['log_comments'] = np.log1p(rq2_df['total_comments'])

# Calculate correlation
correlation, p_value = stats.spearmanr(rq2_df['patch_size'], rq2_df['total_comments'])

# Linear Regression Model (X = log_patch_size, Y = log_comments)
X = sm.add_constant(rq2_df['log_patch_size'])
model = sm.OLS(rq2_df['log_comments'], X).fit()

print("\n--- RQ2 Results ---")
print(f"Spearman Rank Correlation (R): {correlation:.3f}")
print(f"P-value: {p_value:.3f}")
# print(model.summary()) # Full model summary omitted for brevity
print(f"Regression Coefficient (Beta): {model.params['log_patch_size']:.3f}")


--- RQ2 Results ---
Spearman Rank Correlation (R): 0.800
P-value: 0.200
Regression Coefficient (Beta): 0.658


#### RQ1: File-Path Patterns and Acceptance Rate (Descriptive Results)
The analysis examines how the file path category (e.g., src/, docs/, tests/) affects acceptance, comparing Agent-generated PRs (agent=True) against Human-generated PRs (agent=False).

In [3]:
# Calculate acceptance rates per category and agent type
# Outputting the descriptive table
pivot_table = rq1_df.pivot_table(index='path_category', columns='agent', values='acceptance_rate')

print("\n--- RQ1 Descriptive Results (Acceptance Rate) ---")
print(pivot_table.round(3))


--- RQ1 Descriptive Results (Acceptance Rate) ---
agent          False  True 
path_category              
Documentation    0.0    NaN
Source Code      0.0    2.0
Test Files       NaN    1.0
